##Análisis y Predicción de Ventas en una Tienda de Retail (Core)

###Transformación de Datos
- Crea nuevas columnas: Basándonos en los datos existentes, crea nuevas columnas que sean útiles para el análisis. Por ejemplo, calcula el ingreso total por venta y normaliza las ventas.
- Clasifica los datos: Crea una columna que clasifique las ventas en categorías significativas (e.g., ‘Alta’, ‘Media’, ‘Baja’).

In [2]:
import pandas as pd
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/BBDD_Bootcamp_DataScience_Skillnest/retail_sales_dataset.csv"
df = pd.read_csv(path)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    1000 non-null   int64 
 1   Date              1000 non-null   object
 2   Customer ID       1000 non-null   object
 3   Gender            1000 non-null   object
 4   Age               1000 non-null   int64 
 5   Product Category  1000 non-null   object
 6   Quantity          1000 non-null   int64 
 7   Price per Unit    1000 non-null   int64 
 8   Total Amount      1000 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 70.4+ KB


In [5]:
df.head(10)

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30
6,7,2023-03-13,CUST007,Male,46,Clothing,2,25,50
7,8,2023-02-22,CUST008,Male,30,Electronics,4,25,100
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600
9,10,2023-10-07,CUST010,Female,52,Clothing,4,50,200


In [6]:
#Limpiar nombres de columnas
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [8]:
df.columns

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount'],
      dtype='object')

In [10]:
#Crear una columna llamada descuento_aplicado, para anlizar si a la venta tuvo algun descuento aplicado
df['discount_applied'] = (df['quantity'] * df['price_per_unit']) - df['total_amount']
#Crear columna ticket_promedio para analizar cuanto gasta en promedio cada cliente
df['average_ticket_per_customer'] = df.groupby('customer_id')['total_amount'].transform('mean')

In [11]:
df.columns

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount',
       'discount_applied', 'average_ticket_per_customer'],
      dtype='object')

In [20]:
#Clasificar las ventas por "Baja, Media, Alta"
df['sales_level'] = pd.qcut(df['total_amount'], q=3, labels=['baja', 'media', 'alta'])

In [21]:
df.columns

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount',
       'discount_applied', 'average_ticket_per_customer', 'sales_level'],
      dtype='object')

In [23]:
# Saca la columna 'sales_level' del DataFrame y guárdala en una variable para cambiarla de lugar, dejarla al lado de la columna total_amount
col = df.pop('sales_level')
pos = df.columns.get_loc('total_amount') + 1
df.insert(pos, 'sales_level', col)

In [24]:
df.columns

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount',
       'sales_level', 'discount_applied', 'average_ticket_per_customer'],
      dtype='object')

###Agrupación y Agregación
- Agrupación por múltiples columnas: Realiza agrupaciones por categorías como producto y tienda, producto y mes, etc.
- Aplicar funciones de agregación: Utiliza funciones como sum, mean, count, min, max, std, y var para obtener estadísticas descriptivas de cada grupo.

In [25]:
df.head(10)

,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount,sales_level,discount_applied,average_ticket_per_customer
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150,media,0,150.0
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000,alta,0,1000.0
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30,baja,0,30.0
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500,media,0,500.0
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100,media,0,100.0
5,6,2023-04-25,CUST006,Female,45,Beauty,1,30,30,baja,0,30.0
6,7,2023-03-13,CUST007,Male,46,Clothing,2,25,50,baja,0,50.0
7,8,2023-02-22,CUST008,Male,30,Electronics,4,25,100,media,0,100.0
8,9,2023-12-13,CUST009,Male,63,Electronics,2,300,600,alta,0,600.0
9,10,2023-10-07,CUST010,Female,52,Clothing,4,50,200,media,0,200.0


In [26]:
#Agrupaciones
df.groupby("gender")["total_amount"].sum()

,total_amount
gender,
Female,232840
Male,223160


In [27]:
df.groupby(["gender", "product_category"])["total_amount"].sum()

gender  product_category
Female  Beauty              74830
        Clothing            81275
        Electronics         76735
Male    Beauty              68685
        Clothing            74305
        Electronics         80170
Name: total_amount, dtype: int64

In [28]:
df.groupby("gender")["average_ticket_per_customer"].mean().round(2)

,average_ticket_per_customer
gender,
Female,456.55
Male,455.43


In [29]:
df.groupby('product_category')[['quantity', 'total_amount']].sum().reset_index()

,product_category,quantity,total_amount
0,Beauty,771,143515
1,Clothing,894,155580
2,Electronics,849,156905


###Análisis Personalizado con apply
- Función personalizada: Aplica funciones personalizadas para realizar análisis específicos que no se pueden lograr con las funciones de agregación estándar.
- Ejemplo de uso avanzado: Calcula la desviación de cada venta respecto a la media de su grupo.

In [32]:
#Parecido a las agrupaciones y calculos que hice anteriormente
#la diferencia es que aquí agrupo varios calculos dentro de una función y los aplicó con apply
def custom_analysis(group):
    total_qty = group['quantity'].sum()
    total_sales = group['total_amount'].sum()
    total_sales_global = df['total_amount'].sum()
    sales_ratio = (total_sales / total_sales_global).round(2)
    return pd.Series({
        'total_quantity': total_qty,
        'total_sales': total_sales,
        'sales_ratio': sales_ratio
    })

result = df.groupby('product_category').apply(custom_analysis).reset_index()
print(result)

  product_category  total_quantity  total_sales  sales_ratio
0           Beauty           771.0     143515.0         0.31
1         Clothing           894.0     155580.0         0.34
2      Electronics           849.0     156905.0         0.34


<ipython-input-32-2221011811>:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('product_category').apply(custom_analysis).reset_index()


In [36]:
#Calculó de la desviación de cada venta respecto a su media
df['sales_deviation'] = df['total_amount'] - df.groupby('product_category')['total_amount'].transform('mean')
print(df[['product_category', 'total_amount', 'sales_deviation']])

    product_category  total_amount  sales_deviation
0             Beauty           150      -317.475570
1           Clothing          1000       556.752137
2        Electronics            30      -428.786550
3           Clothing           500        56.752137
4             Beauty           100      -367.475570
..               ...           ...              ...
995         Clothing            50      -393.247863
996           Beauty            90      -377.475570
997           Beauty           100      -367.475570
998      Electronics           150      -308.786550
999      Electronics           120      -338.786550

[1000 rows x 3 columns]


In [39]:
df.columns

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount',
       'sales_level', 'discount_applied', 'average_ticket_per_customer',
       'deviation_from_mean', 'sales_deviation'],
      dtype='object')

In [40]:
del df['deviation_from_mean']

In [41]:
df.columns

Index(['transaction_id', 'date', 'customer_id', 'gender', 'age',
       'product_category', 'quantity', 'price_per_unit', 'total_amount',
       'sales_level', 'discount_applied', 'average_ticket_per_customer',
       'sales_deviation'],
      dtype='object')

In [42]:
#Calcular la desviación estandar por grupo
df['std_per_group'] = df.groupby('product_category')['total_amount'].transform('std')
print(df[['product_category', 'total_amount', 'sales_deviation', 'std_per_group']])

    product_category  total_amount  sales_deviation  std_per_group
0             Beauty           150      -317.475570     563.612788
1           Clothing          1000       556.752137     550.695917
2        Electronics            30      -428.786550     567.540150
3           Clothing           500        56.752137     550.695917
4             Beauty           100      -367.475570     563.612788
..               ...           ...              ...            ...
995         Clothing            50      -393.247863     550.695917
996           Beauty            90      -377.475570     563.612788
997           Beauty           100      -367.475570     563.612788
998      Electronics           150      -308.786550     567.540150
999      Electronics           120      -338.786550     567.540150

[1000 rows x 4 columns]
